### Infer single sample

In [336]:
import os, sys
os.environ['PYTHONPATH'] = '/home/qihao/MelodyBot'
sys.path.append('/home/qihao/MelodyBot')
import pickle
import traceback
import random
import subprocess
import torch.cuda
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from utils.earlystopping.protocols import EarlyStopping
from utils.test_dataloder import *
import datetime
from utils.get_time import get_time
import gc, copy
from tqdm import tqdm
from utils.warmup import *
import torch.nn.functional as F
from models.bartmodel_octuple import Bart
from transformers import get_linear_schedule_with_warmup
import prosodic as p

In [337]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [338]:
ckpt_name = 'checkpoint_20240406:190014_lr_5e-05'
# ckpt_name = 'checkpoint_20240406:144930_lr_5e-05'
ckpt_dir = f'/data1/qihao/cs6207/octuple/checkpoints/{ckpt_name}'

In [339]:
src_keys = ['strength', 'length', 'phrase']
tgt_keys = ['bar', 'pos', 'token', 'dur', 'phrase']

binary_dir = '/data1/qihao/cs6207/octuple/binary'
words_dir = '/data1/qihao/cs6207/octuple/binary/words' ## pretrain
# words_dir = '/data1/qihao/cs6207/octuple/binary_909/words' ## 909
hparams = {
    'batch_size': 1,
    'word_data_dir': words_dir,
    'sentence_maxlen': 512,
    'hidden_size': 768,
    'n_layers': 6,
    'n_head': 8,
    'pretrain': '',
    'lr': 5.0e-5,
    'optimizer_adam_beta1': 0.9,
    'optimizer_adam_beta2': 0.98,
    'weight_decay': 0.001,
    'patience': 5,
    'warmup': 2500,
    'lr': 5.0e-5,
    'checkpoint_dir': '/home/qihao/CS6207/octuple/checkpoints',
    'drop_prob': 0.2,
    'total_epoch': 1000,
    'infer_batch_size': 1,
    'temperature': 1.6,
    'topk': 5,
    'prompt_step': 1,
    'infer_max_step': 1024,
    'output_dir': "/home/qihao/CS6207/octuple/output",
}
## tried t=1.6, topk=5, performed best for the PGPR workshop
## tried t=1.3, topk=5, PB: 0.8790575894811564

In [340]:
def set_seed(seed=1234):  # seed setting
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # cuDNN在使用deterministic模式时（下面两行），可能会造成性能下降（取决于model）
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [341]:
def load_model(checkpoint_path, device):
    model = Bart(event2word_dict=event2word_dict, 
                 word2event_dict=word2event_dict, 
                 model_pth='',
                 hidden_size=hparams['hidden_size'], 
                 num_layers=hparams['n_layers'], 
                 num_heads=hparams['n_head'], 
                 dropout=hparams['drop_prob'],).to(device)

    model.load_state_dict(torch.load(checkpoint_path, map_location=device), strict=True)
    model.eval()
    print(f"| Successfully loaded bart ckpt from {checkpoint_path}.")
    return model

In [342]:
def xe_loss(outputs, targets):
    outputs = outputs.transpose(1, 2)
    return F.cross_entropy(outputs, targets, ignore_index=0, reduction='mean')

In [343]:
def convert_to_data(data_sample, event2word_dict, word2event_dict):   
    data = {}
    for key, value in data_sample.items():
        end_name, dict_key = key.split('_')
        if dict_key.lower() == 'token':
            dict_key = 'Pitch'
        else:
            dict_key = dict_key[0].upper() + dict_key[1:].lower()
        input_tokens = []
        for v in value:
            # print(dict_key)
            input_tokens.append(event2word_dict[dict_key][v])
        data[key] = torch.LongTensor([copy.deepcopy(input_tokens)])

    ## tgt_input:

    '''
    data = {
        'src_strength': torch.LongTensor([[5, 3, 5, 3, 5, 3]]),
        'src_length': torch.LongTensor([[3, 3, 4, 4, 4, 3]]),
        'src_phrase': torch.LongTensor([[4, 4, 4, 4, 4, 3]]),
        'tgt_bar': torch.LongTensor([[1, 3]]),
        'tgt_pos': torch.LongTensor([[1, 75]]),
        'tgt_token': torch.LongTensor([[1, 63]]),
        'tgt_dur': torch.LongTensor([[1, 22]]),
        'tgt_phrase': torch.LongTensor([[1, 4]]),
    }
    '''
    return data

In [344]:
def line_to_prosody(line):
    length = len(line)
    
    # Predefined strength patterns for lengths 2 to 7
    s_patterns = {
        2: ['<strong>', '<weak>'],
        3: ['<strong>', '<weak>', '<weak>'],
        4: ['<strong>', '<weak>', '<strong>', '<weak>'],
        5: ['<strong>', '<weak>', '<strong>', '<weak>', '<strong>'],
        7: ['<strong>', '<weak>', '<strong>', '<weak>', '<weak>', '<weak>', '<strong>'],
    }
    
    # Strength pattern generation for lengths 6 to 12
    if length in s_patterns:
        strength = s_patterns[length]
    elif length % 2 == 0:
        # Even length >= 6: alternate strong and weak, starting with strong
        strength = ['<strong>' if i % 2 == 0 else '<weak>' for i in range(length)]
    else:
        # Odd length > 7: strong at 1st, 3rd, 7th positions (0-based), rest weak
        strength = []
        for i in range(length):
            if i == 0 or i == 2 or i == 6:
                strength.append('<strong>')
            else:
                strength.append('<weak>')
    
    # Length pattern: all <short> except last <long>
    length_pattern = ['<short>'] * (length - 1) + ['<long>']
    
    # Phrase end pattern: all <false> except last <true>
    phrase_end = ['<false>'] * (length - 1) + ['<true>']
    
    return strength, length_pattern, phrase_end

In [345]:
import prosodic as p
def convert_lyrics_to_input (lyrics, S, L, P):
    # text = p.Text(lyrics)

    s5 = ['<strong>', '<weak>', '<strong>', '<weak>', '<strong>']
    l5 = ['<short>', '<short>', '<short>', '<short>', '<long>']
    s7 = ['<strong>', '<weak>', '<strong>', '<weak>', '<weak>', '<weak>', '<strong>']
    l7 = ['<short>', '<short>', '<short>', '<short>', '<short>', '<short>', '<long>']
    p5 = ['<false>', '<false>', '<false>', '<false>', '<true>']
    p7 = ['<false>', '<false>', '<false>', '<false>', '<false>', '<false>', '<true>']
    
    rep = 10
    data_sample = {
        # 'src_strength': ['<strong>', '<weak>', '<strong>', '<weak>', '<weak>', '<weak>', '<strong>']*rep*4,
        # 'src_length': ['<short>', '<short>', '<short>', '<short>', '<short>', '<short>', '<long>']*rep*4,
        # 'src_strength': s5*49+s7*72,
        # 'src_length': l5*49+l7*72,
        # 'src_phrase': p5*49+p7*72,
        'src_strength': S,
        'src_length': L,
        'src_phrase': P,
        'tgt_bar': ["<s>", "Bar_0"],
        'tgt_pos': ["<pad>", "Pos_0"],
        'tgt_token': ["<pad>", "Pitch_60"],
        'tgt_dur': ["<pad>", "Dur_120"],
        'tgt_phrase': ["<pad>", "<false>"],
    }
    
    """
    for line_id, line in enumerate(text.lines()):
        words = line.words()
        line_syllables = line.syllables()
        line_syllable_num = len(line_syllables)

        bound = '<false>'
        
        
        ### src words
        for syl_id, s in enumerate(line_syllables):
            print(s, end='  ')
            ## is accented:
            if "'" in str(s): ## strong
                mtype = "<strong>"
            elif "`" in str(s):
                mtype = "<substrong>"
            else:
                mtype = "<weak>"
            length = "<long>" if "ː" in str(s) else "<short>"
            # data_sample['src_strength'].append(mtype)
            # data_sample['src_length'].append(length)
            '''
            if syl_id == len(line_syllables)-1:
                data_sample['src_phrase'].append('<true>')
            else:
                data_sample['src_phrase'].append('<false>')
            '''
    """

    # data_sample['src_strength'] = data_sample['src_strength'] * 5
    # data_sample['src_length'] = data_sample['src_length'] * 6
    # data_sample['src_phrase'] = data_sample['src_phrase'] * rep
    return data_sample

In [346]:
char_lyrics = '''Hey Jude don't make it bad
Take a sad song and make it better
Remember to let her into your heart
Then you can start to make it better
Hey Jude don't be afraid
You were made to go out and get her
The minute you let her under your skin
Then you begin to make it better
And anytime you feel the pain
hey Jude refrain
Don't carry the world upon your shoulders
For well you know that it's a fool who plays it cool
By making his world a little colder
'''

In [347]:
def infer_l2m(data_sample, output_dir='./', song_name='0'):
    ## -------------------
    ##     Bart Model
    ## -------------------
    set_seed()
    print(f"Using device: {device} for inferences custom samples")
    
    # training conditions (for naming the ckpt)
    lr = hparams['lr']
    
    ckpt_path = os.path.join(ckpt_dir, 'best.pt')

    # load dictionary
    event2word_dict, word2event_dict = pickle.load(open(f"{binary_dir}/music_dict.pkl", 'rb'))

    # load melody generation model based on skeleton framework
    model = Bart(event2word_dict=event2word_dict, 
                 word2event_dict=word2event_dict, 
                 model_pth='',
                 hidden_size=hparams['hidden_size'], 
                 num_layers=hparams['n_layers'], 
                 num_heads=hparams['n_head'], 
                 dropout=hparams['drop_prob'],).to(device)
    model.load_state_dict(torch.load(ckpt_path, map_location=device), strict=True)
    model.eval()
    print(f"| Successfully loaded bart ckpt from {ckpt_path}.")

    # -------------------------------------------------------------------------------------------
    # Inference file path
    # -------------------------------------------------------------------------------------------
    # exp_date = get_time()
    # melody_output_dir = os.path.join(hparams['output_dir'], f'{song_name}')
    # melody_output_dir = f'./{song_name}'
    # melody_output_dir = os.path.join(output_dir, song_name)
    melody_output_dir = output_dir
    if not os.path.exists(melody_output_dir):
        os.mkdir(melody_output_dir)


    data = convert_to_data(data_sample, event2word_dict, word2event_dict)
    print(data)
    
    try:
        data_name = song_name

        enc_inputs = {k: data[f'src_{k}'].to(device) for k in src_keys}
        dec_inputs = {k: data[f'tgt_{k}'].to(device) for k in tgt_keys}
        
        print(enc_inputs['strength'].shape)

        prompt_step = len(data_sample['tgt_bar'])

        dec_inputs_selected = {
            'bar': dec_inputs['bar'][:, :prompt_step],
            'pos': dec_inputs['pos'][:, :prompt_step],
            'token': dec_inputs['token'][:, :prompt_step],
            'dur': dec_inputs['dur'][:, :prompt_step],
            'phrase': dec_inputs['phrase'][:, :prompt_step],
        }
        print(enc_inputs)
        print(dec_inputs_selected)

        decode_length = enc_inputs['strength'].shape[-1]+2
        max_sent_len = 1024  # 1024

        print(f"Expected decode length: {decode_length}")
        _ = model.infer(enc_inputs=enc_inputs, 
                        dec_inputs_gt=dec_inputs_selected, 
                        decode_length=decode_length-prompt_step,
                        sentence_maxlen=max_sent_len, 
                        temperature=hparams['temperature'], 
                        topk=hparams['topk'], 
                        device=device, 
                        output_dir=melody_output_dir, 
                        midi_name=data_name)

        # print(f"Generating {data_idx+1}/{len(test_loader)}, Name: {data_name}")
    except Exception as e:
        traceback.print_exc()
        print(f"-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-\nBad Item: {data_name}")

In [ ]:
dataset = '/home/qihao/muzic/telemelody/inference/results/poems_dataset.txt'
with open(dataset, 'r') as ds:
    all_lines = ds.read().split()
print(len(all_lines))
ds.close()

song_id = 0
step = 90
for ts in range(0, len(all_lines), step):
    S, L, P = [], [], []
    if ts + step <= len(all_lines):
        for idx in range(ts, ts+step):
            line = all_lines[idx]
            s, l, p = line_to_prosody(line)
            S.extend(s.copy())
            L.extend(l.copy())
            P.extend(p.copy())
    else:
        for idx in range(ts, len(all_lines)):
            line = all_lines[idx]
            s, l, p = line_to_prosody(line)
            S.extend(s.copy())
            L.extend(l.copy())
            P.extend(p.copy())
    print(len(S), len(L), len(P))
    sample = convert_lyrics_to_input(char_lyrics, S.copy(), L.copy(), P.copy())
    infer_l2m(sample, output_dir='/home/qihao/MelodyBot/3_inference/Output/workshop', song_name=str(song_id))
    song_id = song_id + 1

1516
28 28 28
Using device: cuda:1 for inferences custom samples


  1%|          | 8/1024 [00:00<00:13, 77.21it/s]

| Successfully loaded bart ckpt from /data1/qihao/cs6207/octuple/checkpoints/checkpoint_20240406:190014_lr_5e-05/best.pt.
{'src_strength': tensor([[3, 5, 3, 5, 5, 5, 3, 3, 5, 3, 5, 5, 5, 3, 3, 5, 3, 5, 5, 5, 3, 3, 5, 3,
         5, 5, 5, 3]]), 'src_length': tensor([[4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4,
         4, 4, 4, 3]]), 'src_phrase': tensor([[4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4,
         4, 4, 4, 3]]), 'tgt_bar': tensor([[0, 3]]), 'tgt_pos': tensor([[1, 3]]), 'tgt_token': tensor([[ 1, 63]]), 'tgt_dur': tensor([[1, 8]]), 'tgt_phrase': tensor([[1, 4]])}
torch.Size([1, 28])
{'strength': tensor([[3, 5, 3, 5, 5, 5, 3, 3, 5, 3, 5, 5, 5, 3, 3, 5, 3, 5, 5, 5, 3, 3, 5, 3,
         5, 5, 5, 3]], device='cuda:1'), 'length': tensor([[4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4,
         4, 4, 4, 3]], device='cuda:1'), 'phrase': tensor([[4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 

  5%|▌         | 55/1024 [00:00<00:12, 76.01it/s]


Decode ends at step 55
/home/qihao/MelodyBot/3_inference/Output/workshop/0.mid
28 28 28
Using device: cuda:1 for inferences custom samples


...

Decode ends at step 26
/home/qihao/MelodyBot/3_inference/Output/workshop/377.mid
20 20 20
Using device: cuda:1 for inferences custom samples


  1%|          | 10/1024 [00:00<00:11, 91.54it/s]

| Successfully loaded bart ckpt from /data1/qihao/cs6207/octuple/checkpoints/checkpoint_20240406:190014_lr_5e-05/best.pt.
{'src_strength': tensor([[3, 5, 3, 5, 3, 3, 5, 3, 5, 3, 3, 5, 3, 5, 3, 3, 5, 3, 5, 3]]), 'src_length': tensor([[4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3]]), 'src_phrase': tensor([[4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3]]), 'tgt_bar': tensor([[0, 3]]), 'tgt_pos': tensor([[1, 3]]), 'tgt_token': tensor([[ 1, 63]]), 'tgt_dur': tensor([[1, 8]]), 'tgt_phrase': tensor([[1, 4]])}
torch.Size([1, 20])
{'strength': tensor([[3, 5, 3, 5, 3, 3, 5, 3, 5, 3, 3, 5, 3, 5, 3, 3, 5, 3, 5, 3]],
       device='cuda:1'), 'length': tensor([[4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3]],
       device='cuda:1'), 'phrase': tensor([[4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 4, 3]],
       device='cuda:1')}
{'bar': tensor([[0, 3]], device='cuda:1'), 'pos': tensor([[1, 3]], device='cuda:1'), 'token': tensor([[ 1, 63]], device='cud

  3%|▎         | 26/1024 [00:00<00:11, 87.82it/s]

Decode ends at step 26
/home/qihao/MelodyBot/3_inference/Output/workshop/378.mid


In [349]:
def generate (lyrics_in):
    test_sample = convert_lyrics_to_input(lyrics_in)
    infer_l2m(sample, output_dir='./')

In [350]:
# generate('')